# Extract relevant climate data for the Uvira region

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import climateserv as cs

from io import StringIO
import requests
import json
import pprint

In [2]:
raw_data_dir = Path("../../data/raw")

## General Parameters

In [3]:
x_center = 29.147
y_center = -3.37
dx = 0.05
dy = 0.05

GeometryCoords = [
    [x_center - dx, y_center + dy],
    [x_center + dx, y_center + dy],
    [x_center + dx, y_center - dy], 
    [x_center - dx, y_center - dy],
    [x_center - dx, y_center + dy]
]
operation = "Average"
start_date = "03/01/2016"
end_date = "01/31/2025"
seasonal_ensemble = '' # Leave empty when using the new integer dataset IDs
seasonal_variable = ''

DAHITI_KEY = "696584C92981A2B3DCA7AED9B7A5B29EB9212A5F7C16DCE3E2CE9EF141C5396D"

## Rainfall Data

Data from climateserv python api: https://github.com/SERVIR/ClimateSERVpy

More information: https://chc.ucsb.edu/data/chirps3

In [4]:
dataset_rain = 90
outfile_rain = str(raw_data_dir / "rainfall.csv")

In [5]:
if Path(outfile_rain).exists():
    Path(outfile_rain).unlink()
cs.api.request_data(
    data_set_type = dataset_rain, 
    operation_type = operation, 
    earliest_date = start_date,
    latest_date = end_date,
    geometry_coords = GeometryCoords,
    seasonal_ensemble = seasonal_ensemble,
    seasonal_variable = seasonal_variable,
    outfile = outfile_rain
)

New Script Run, Dataset: 90
Data request submitted. Request ID: 7fc4bbc2-f50c-4072-bd0b-a4d59268f532
Progress: 2.8333333333333335
Progress: 8.5
Progress: 19.833333333333332
Progress: 33.99999999999999
Progress: 42.5
Progress: 53.83333333333334
Progress: 68.00000000000001
Progress: 76.5
Progress: 82.16666666666666
Data request is complete.
Attempting to write CSV Data to: ../../data/raw/rainfall.csv
CSV Data Written to: ../../data/raw/rainfall.csv
Exiting...


## Surface Soil Moisture

Data from climateserv python api: https://github.com/SERVIR/ClimateSERVpy

More information from: https://earth.gsfc.nasa.gov/hydro/data/nasa-usda-global-soil-moisture-data


In [8]:
dataset_ssm = 38
outfile_ssm = str(raw_data_dir / "surface_soil_moisture.csv")

In [9]:
if Path(outfile_ssm).exists():
    Path(outfile_ssm).unlink()
cs.api.request_data(
    data_set_type = dataset_ssm, 
    operation_type = operation, 
    earliest_date = start_date,
    latest_date = end_date,
    geometry_coords = GeometryCoords,
    seasonal_ensemble = seasonal_ensemble,
    seasonal_variable = seasonal_variable,
    outfile = outfile_ssm
)

New Script Run, Dataset: 38
Data request submitted. Request ID: 10f25d55-532c-47dd-b89e-faa2123995e5
Progress: 4.261904761904762
Progress: 34.095238095238095
Progress: 63.928571428571416
Progress: 89.49999999999997
Data request is complete.
Attempting to write CSV Data to: ../../data/raw/surface_soil_moisture.csv
CSV Data Written to: ../../data/raw/surface_soil_moisture.csv
Exiting...


## Sub-Surface Soil Moisture

Data from climateserv python api: https://github.com/SERVIR/ClimateSERVpy

More information from: https://earth.gsfc.nasa.gov/hydro/data/nasa-usda-global-soil-moisture-data

In [10]:
dataset_subssm = 40
outfile_subssm = str(raw_data_dir / "sub_surface_soil_moisture.csv")

In [11]:
if Path(outfile_subssm).exists():
    Path(outfile_subssm).unlink()
cs.api.request_data(
    data_set_type = dataset_subssm, 
    operation_type = operation, 
    earliest_date = start_date,
    latest_date = end_date,
    geometry_coords = GeometryCoords,
    seasonal_ensemble = seasonal_ensemble,
    seasonal_variable = seasonal_variable,
    outfile = outfile_subssm
)

New Script Run, Dataset: 40
Data request submitted. Request ID: 5d0935a4-bcd3-4884-a64b-a877d9bc1372
Progress: 4.261904761904762
Progress: 34.095238095238095
Progress: 59.66666666666665
Progress: 89.49999999999997
Data request is complete.
Attempting to write CSV Data to: ../../data/raw/sub_surface_soil_moisture.csv
CSV Data Written to: ../../data/raw/sub_surface_soil_moisture.csv
Exiting...


## Evaporative Stress Index

info at: https://gis1.servirglobal.net/geonetwork/srv/eng/catalog.search#/metadata/97082c3e-a136-4689-bcdf-517a4bb6907f

In [12]:
dataset_esi = 29
outfile_esi = str(raw_data_dir / "evaporative_stress_index.csv")

In [13]:
if Path(outfile_esi).exists():
    Path(outfile_esi).unlink()
cs.api.request_data(
    data_set_type = dataset_esi, 
    operation_type = operation, 
    earliest_date = start_date,
    latest_date = end_date,
    geometry_coords = GeometryCoords,
    seasonal_ensemble = seasonal_ensemble,
    seasonal_variable = seasonal_variable,
    outfile = outfile_esi
)

New Script Run, Dataset: 29
Data request submitted. Request ID: e11367e8-9ea2-4848-a842-43fc9ea445c3
Progress: 2.8333333333333335
Progress: 22.666666666666664
Progress: 42.5
Progress: 56.66666666666668
Progress: 76.5
Progress: 84.99999999999999
Data request is complete.
Attempting to write CSV Data to: ../../data/raw/evaporative_stress_index.csv
CSV Data Written to: ../../data/raw/evaporative_stress_index.csv
Exiting...


## Water Level Hypsometry

In [14]:
outfile_lake = raw_data_dir / "tanganyika_water_level.csv"

In [15]:
if outfile_lake.exists():
    outfile_lake.unlink()

url = "https://dahiti.dgfi.tum.de/api/v2/download-water-level/"

args = {}
args['api_key'] = DAHITI_KEY
args['dahiti_id'] = 25
args['format'] = 'csv'

response = requests.get(url, params=args)

if response.status_code == 200:
	data_csv = response.text
	(
		pd.read_csv(
			StringIO(data_csv),
			parse_dates=['datetime'],
			sep=";"
		)
		.rename(columns={'datetime': 'date'})
		.loc[lambda x: (
			(x["date"] >= pd.Timestamp(start_date))
			& (x["date"] <= pd.Timestamp(end_date))
		)]
		[['date', 'water_level']]
		.to_csv(outfile_lake, index=False)
	)
	print("done!")			
else:
	print (response.text)
	print (response.status_code)

done!
